<a href="https://colab.research.google.com/github/giancampo/digital-analytics-python-scripts/blob/main/AnalyticsFTW_GA4UtmTagsClassifier_v0_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import python libraries



In [ ]:
import pandas as pd
import re

#import GA4 source categories file
(and filter sources by categories)

In [ ]:
# Load the uploaded Excel file
file_path = '/content/NK_AnalyticsFTW_GA4SourceCategories.xlsx'

# Assuming the values are in the first column of the first sheet, we'll read that column
values_df = pd.read_excel(file_path)

# Filtering the values from the Excel file where 'source category' is 'SOURCE_CATEGORY_SHOPPING'
filtered_shopping_sources = values_df[values_df['source category'] == 'SOURCE_CATEGORY_SHOPPING']['source'].tolist()
# Filtering the values from the Excel file where 'source category' is 'SOURCE_CATEGORY_SEARCH'
filtered_search_sources = values_df[values_df['source category'] == 'SOURCE_CATEGORY_SEARCH']['source'].tolist()
# Filtering the values from the Excel file where 'source category' is 'SOURCE_CATEGORY_SOCIAL'
filtered_social_sources = values_df[values_df['source category'] == 'SOURCE_CATEGORY_SOCIAL']['source'].tolist()
# Filtering the values from the Excel file where 'source category' is 'SOURCE_CATEGORY_VIDEO'
filtered_video_sources = values_df[values_df['source category'] == 'SOURCE_CATEGORY_VIDEO']['source'].tolist()

# Display the first few rows of the DataFrame to understand its structure
#values_df.head()

#upload the file to check
(Excel file with columns exactly named: "Source", "Medium", "Campaign". The file must be named "Check.xlsx")

In [ ]:
uploaded_file_path = '/content/Check.xlsx'
df = pd.read_excel(uploaded_file_path, na_filter=False)
#df.head()

#map the input UTM tags
(and define their default channel groups)

In [ ]:
'''
# Sample data to test the script
data = {
    'Source': ['(direct)', 'google', 'direct', 'facebook', 'alibaba', 'test', 'foo', 'foo'],
    'Medium': ['(not set)', 'cp', 'interstitial', '(none)', 'foo', 'cp', 'retargeting', 'sm'],
    'Campaign': ['foo', 'foo', 'foo', 'cross-network', 'foo', 'shop', 'bar', 'foo']
}

df = pd.DataFrame(data)
'''

# Define the regex patterns
# Some of them are duplicates but they could change in the future
paid_shopping_campaign_pattern = r'^(.*(([^a-df-z]|^)shop|shopping).*)$'
paid_shopping_medium_pattern = r'^(.*cp.*|ppc|retargeting|paid.*)$'
paid_search_medium_pattern = r'^(.*cp.*|ppc|retargeting|paid.*)$'
paid_social_medium_pattern = r'^(.*cp.*|ppc|retargeting|paid.*)$'
paid_video_medium_pattern = r'^(.*cp.*|ppc|retargeting|paid.*)$'
paid_other_medium_pattern = r'^(.*cp.*|ppc|retargeting|paid.*)$'
organic_shopping_campaign_pattern = r'^(.*(([^a-df-z]|^)shop|shopping).*)$'
organic_video_medium_pattern = r'^(.*video.*)$'
email_source_pattern = r'email|e-mail|e_mail|e mail'
email_medium_pattern = r'email|e-mail|e_mail|e mail'
mobile_medium_pattern_1 = r'.*push$'
mobile_medium_pattern_2 = r'.*mobile.*|.*notification.*'

# Function to apply the condition
def check_condition(row):
    Source = row['Source'].lower()
    Medium = row['Medium'].lower()
    Campaign = row['Campaign'].lower()
    if Source == '(direct)' and Medium in ['(not set)', '(none)']:
        return 'Direct'
    elif Campaign == 'cross-network':
        return 'Cross-network'
    elif Source in filtered_shopping_sources or (re.match(paid_shopping_campaign_pattern, Campaign) and re.match(paid_shopping_medium_pattern, Medium)):
        return 'Paid Shopping'
    elif Source in filtered_search_sources and re.match(paid_search_medium_pattern, Medium):
        return 'Paid Search'
    elif Source in filtered_social_sources and re.match(paid_social_medium_pattern, Medium):
        return 'Paid Social'
    elif Source in filtered_video_sources and re.match(paid_video_medium_pattern, Medium):
        return 'Paid Video'
    elif Medium in ['display', 'banner', 'expandable', 'interstitial', 'cpm']:
        return 'Display'
    elif re.match(paid_other_medium_pattern, Medium):
        return 'Paid Other'
    elif Source in filtered_shopping_sources or re.match(organic_shopping_campaign_pattern, row['Campaign']):
        return 'Organic Shopping'
    elif Source in filtered_social_sources or Medium in ['social', 'social-network', 'social-media', 'sm', 'social network', 'social media']:
        return 'Organic Social'
    elif Source in filtered_video_sources or re.match(organic_video_medium_pattern, Medium):
        return 'Organic Video'
    elif Source in filtered_search_sources or Medium == 'organic':
        return 'Organic Search'
    elif Medium in ['referral', 'app', 'link']:
        return 'Referral'
    elif re.match(email_source_pattern, Source) or re.match(email_medium_pattern, Medium):
        return 'Email'
    elif Medium == 'affiliate':
        return 'Affiliates'
    elif Medium == 'audio':
        return 'Audio'
    elif Source == 'sms' or Medium == 'sms':
        return 'SMS'
    elif re.match(mobile_medium_pattern_1, Medium) or re.match(mobile_medium_pattern_2, Medium) or Source == 'firebase':
        return 'Mobile Push Notifications'
    else:
        return 'Unassigned'  # Or any other default value you wish to keep

# Apply the function to each row
df['Default channel group'] = df.apply(check_condition, axis=1)

# Display the DataFrame
#df.head()

# Export the DataFrame
df.to_excel("results.xlsx", sheet_name='UTM tagging classification', index=False)
